In [1]:
///\section*{What's new}
///\begin{itemize}
/// \item implementing the Lax-Friedrichs flux
/// \item computation of "experimental order of convergence" (EOC)
///\end{itemize}
///
///\section*{Prerequisites}
///\begin{itemize}
/// \item definition of fluxes, chapter \ref{SpatialOperator}
/// \item grid generation, chapter \ref{GridInstantiation}
/// \item basics of convergence studies, chapter \ref{GridInstantiation}
///\end{itemize}
///Within this tutorial the Lax-Friedrichs flux will be implemented as an alternative to the Upwind flux (see chapter \ref{SpatialOperator}). 
/// For both fluxes a convergence study will be performed and the "experimental order of convergence" (EOC) will be computed. 
///For the implementation of the Lax-Friedrichs flux and the Upwind flux it is recommended to work through chapter \ref{SpatialOperator} first 
/// as the definition of fluxes is explained there in more detail. For the second part of the tutorial, 
/// chapter \ref{GridInstantiation} has already taught the basics of doing a convergence study.
///
///\section{Problem statement}
///\label{sec:NumFlux_problem}
///As an examplary problem, we consider the scalar transport equation
///\begin{equation}
///\frac{\partial c}{\partial t} + \nabla \cdot (\vec{u} c) = 0
///\end{equation}
///in the domain $\domain = [-1, 1] \times [-1, 1]$, where the concentration $c = c(x,y,t) \in \mathbb{R}$ is unknown and the velocity field is given by
///\begin{equation*}
///\vec{u} = \begin{pmatrix}
///y\\-x
///\end{pmatrix}
///\end{equation*}
///The analytical solution for the problem above is given by 
///\begin{align*}
///c_\text{Exact}(x,y,t) &= \cos(\cos(t) x - \sin(t) y) \quad \text{ for } (x,y) \in \domain,
///\end{align*}
///which can be used to describe the initial and boundary conditions. 
///
///\section{Solution within the BoSSS Framework}
///\label{sec:NumFlux_tutorial}

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [3]:
/// First, we define functions for the given velocity field and the exact solution

In [4]:
public static class MyGlobals {
    public static double u(double[] X) => X[1];
    public static double v(double[] X) => -X[0];
    public static double cExact(double[] X, double t) => Math.Cos(Math.Cos(t)*X[0] - Math.Sin(t)*X[1]);
}

In [5]:
/// %=============================================
/// \subsection{Definition of the numerical fluxes} \label{sec:NumFlux_fluxes}
/// \subsection{Definition of the numerical fluxes} \label{sec:flux_and_convergence_fluxes}
/// %=============================================

In [6]:
using BoSSS.Platform.LinAlg;

In [7]:
/// Before implementing the \code{ScalarTransportFlux} as done in Tutorial 4, we define both fluxes 
/// as functions in terms of the parameters \code{Uin}, \code{Uout}, the normal vector \code{n} and the
/// \code{velocityVector}. Recalling the Upwind flux, the corresponding flux function is defined as

In [8]:
Func<double, double, Vector, Vector, double> upwindFlux =     
    delegate(double Uin, double Uout, Vector n, Vector velocityVector) {     
 
        if (velocityVector * n > 0) {     
            return (velocityVector * Uin) * n;     
        } else {     
           return (velocityVector * Uout) * n;     
        }     
    };

In [9]:
/// The second flux we are considering now, is the Lax-Friedrichs flux
/// \begin{equation*}
/// \hat{f}(c_h^-, c_h^+, \vec{u} \cdot \vec{n}) = \mean{\vec{u} \, c_h} \cdot \vec{n} + \frac{C}{2} \jump{c_h}.
/// \end{equation*}
/// The constant $C \in \mathbb{R}^+$ has to be sufficiently large in order to guarantee the stability of the 
/// numerical scheme. We choose 
/// \begin{equation*}
/// C = \max \abs{\vec{u} \cdot \vec{n}} = 1
/// \end{equation*}
/// for the given problem.

In [10]:
double C = 0.3;     
 
Func<double, double, Vector, Vector, double> laxFriedrichsFlux =     
    delegate(double Uin, double Uout, Vector n, Vector velocityVector) {     
 
        return 0.5 * (Uin + Uout) * velocityVector * n - C * (Uout - Uin);     
    };

In [11]:
/// For the implementation of the \code{ScalarTransportFlux} we want to generate 
/// instances by the definition of the \code{numericalFlux}. Therefore a new constructor is implemented 
/// and the implementation of \code{InnerEdgeFlux} is rewritten in terms of the \code{numericalFlux}.
class ScalarTransportFlux : NonlinearFlux {     
 
    private Func<double, double, Vector, Vector, double> numericalFlux;     
 
    // Provides instances of this class with a specific flux implementation     
    public ScalarTransportFlux(     
        Func<double, double, Vector, Vector, double> numericalFlux) {     
 
        this.numericalFlux = numericalFlux;     
    }     
 
    public override IList<string> ArgumentOrdering {     
        get { return new string[] { "ch" }; }     
    }     
 
    protected override void Flux(     
        double time, double[] x, double[] U, double[] output) {     
 
        output[0] = MyGlobals.u(x) * U[0];     
        output[1] = MyGlobals.v(x) * U[0];     
    }     
 
    // Makes use of the flux implementation supplied in the constructor     
    protected override double InnerEdgeFlux(     
        double time, double[] x, double[] normal,      
        double[] Uin, double[] Uout, int jEdge) {     
 
        Vector n              = new Vector(normal);     
        Vector velocityVector = new Vector(MyGlobals.u(x), MyGlobals.v(x));     
 
        return numericalFlux(Uin[0], Uout[0], n, velocityVector);     
    }     
 
    protected override double BorderEdgeFlux(     
        double time, double[] x, double[] normal, byte EdgeTag,     
        double[] Uin, int jEdge) {     
 
        double[] Uout = new double[] { MyGlobals.cExact(x, time) };     
 
        return InnerEdgeFlux(time, x, normal, Uin, Uout, jEdge);     
    }     
}

In [12]:
/// For both numerical fluxes we define a \code{SpatialOperator} that uses the corresponding flux to compute div(...)

In [13]:
var upwindOperator = new SpatialOperator(     
    new string[] { "ch" },     
    new string[] { "div" },     
    QuadOrderFunc.NonLinear(2));

In [14]:
upwindOperator.EquationComponents["div"].Add(     
    new ScalarTransportFlux(upwindFlux));

In [15]:
upwindOperator.Commit();

In [16]:
/// %

In [17]:
var laxFriedrichsOperator = new SpatialOperator(     
    new string[] { "ch" },     
    new string[] { "div" },     
    QuadOrderFunc.NonLinear(2));

In [18]:
laxFriedrichsOperator.EquationComponents["div"].Add(     
    new ScalarTransportFlux(laxFriedrichsFlux));

In [19]:
laxFriedrichsOperator.Commit();

In [20]:
/// %

In [21]:
/// %=============================================
/// \subsection{Convergence study}\label{sec:flux_and_convergence_convergence}
/// %=============================================

In [22]:
/// In the following a convergence study for the discretization error will be performed on grids 
/// with $2\times2$, $4\times4$, $8\times8$ and $16\times16$ cells. The error at $t = \pi /4$
/// will be investigated for the polynomial degrees $\degree = 0, \ldots, 3$ of the ansatzfunctions. 
/// The study will be done for both numerical fluxes.

In [23]:
/// We start by defining a series of nested grids for the convergence study

In [24]:
int[] numbersOfCells = new int[] { 2, 4, 8, 16 };

In [25]:
GridData[] grids = new GridData[numbersOfCells.Length];

In [26]:
for (int i = 0; i < numbersOfCells.Length; i++) {     
 
    double[] nodes = GenericBlas.Linspace(-1.0, 1.0, numbersOfCells[i] + 1);     
 
    GridCommons grid = Grid2D.Cartesian2DGrid(nodes, nodes);     
 
    grids[i] = new GridData(grid);     
}

In [27]:
/// Then, a \code{SinglePhaseField} is defined for each polynomial degree on each grid.
/// The initial value is projected on each field.

In [28]:
int[] dgDegrees = new int[] { 0, 1, 2, 3 };

In [29]:
SinglePhaseField[,] fields =      
    new SinglePhaseField[dgDegrees.Length, numbersOfCells.Length];

In [30]:
for (int i = 0; i < dgDegrees.Length; i++) {     
    for (int j = 0; j < numbersOfCells.Length; j++) {     
 
        Basis basis = new Basis(grids[j], dgDegrees[i]);     
        fields[i, j] = new SinglePhaseField(     
                              basis, "ch_" + dgDegrees[i] + "_" + grids[j]);     
        fields[i, j].ProjectField(X => MyGlobals.cExact(X, 0.0));     
    }     
}

In [31]:
/// Since the error at a fourth revolution is considered, the initial concentration has to be simulated until 
/// $\code{endTime} = \pi / 4$. For the timestepping we are using the classical fourth order Runge-Kutta scheme. 
/// The timestep size should be chosen small enough in order to reduce the \emph{temporal} error,
/// so that the \emph{spatial} error dominates for the convergence study.

In [32]:
using BoSSS.Solution.Timestepping;

In [33]:
double endTime        = Math.PI / 4.0;     
int numberOfTimesteps = 100;     
double dt             = endTime / numberOfTimesteps;

In [34]:
/// A \code{MultidimensionalArray} is a special double array that enables \emph{shallow} resizing and data
/// extraction. For example, sub-arrays can be extracted without needing to copy the entries of
/// the \code{MultidimensionalArray}. BoSSS makes extensive use of this class because this is crucial
/// for the performance.
/// Here, we create a three-dimensional array, where the first index corresponds to the flux
/// function, the second to the DG degree and the third to the number of cells.
MultidimensionalArray errors =      
    MultidimensionalArray.Create(2, dgDegrees.Length, grids.Length);

In [35]:
for (int i = 0; i < dgDegrees.Length; i++) {     
    for (int j = 0; j < numbersOfCells.Length; j++) {     
 
        // Clones an object and casts it to the type of the original object  
        // at the same time.  
        SinglePhaseField c = fields[i, j].CloneAs();     
 
        // Instantiate the timestepper (classical Runge-Kutta scheme)     
        // for the evolution of the concentration c with Upwind flux     
        RungeKutta timeStepper = new RungeKutta(     
            RungeKutta.RungeKuttaSchemes.RungeKutta1901,     
            upwindOperator, c);     
 
        // Integrate in time for each timestep     
        for (int k = 0; k < numberOfTimesteps; k++) {     
            timeStepper.Perform(dt);     
        }     
 
        // Compute the error w.r.t. analytical solution     
        errors[0, i, j] = c.L2Error(X => MyGlobals.cExact(X, endTime));     
 
        // Simulate with the Lax-Friedrichs flux     
        c = fields[i, j].CloneAs();     
        timeStepper = new RungeKutta(     
                             RungeKutta.RungeKuttaSchemes.RungeKutta1901,      
                             laxFriedrichsOperator, c);     
        for (int k = 0; k < numberOfTimesteps; k++) {     
            timeStepper.Perform(dt);     
        }     
        errors[1, i, j] = c.L2Error(X => MyGlobals.cExact(X, endTime));     
    }     
}

In [36]:
/// %=============================================
/// \subsection{Plotting results}\label{sec:NumFlux_plots}
/// \subsection{Plotting results}\label{sec:flux_and_convergence_plots}
/// %=============================================

In [37]:
/// For the convergence plots the error is plotted over the mesh size, 
/// where the coarsest grid size is defined as size = 1.

In [38]:
var sizes = numbersOfCells.Select(s => 2.0 / s).ToArray();

In [39]:
/// First, we are looking at the convergence plot of the Upwind flux. 
/// Therefore, an instance of Gnuplot has to be generated.
var gpUpwind = new Gnuplot();     
gpUpwind.SetYRange(Math.Pow(10,-7), Math.Pow(10,0));

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


In [40]:
for (int i = 0; i < dgDegrees.Length; i++) {     
 
    /// Here, we use the shallow extraction features of    
    /// \code{MultidimensionalArray} mentioned above.   
    /// The command \code{ExtractSubArrayShallow(0, i, -1)}    
    /// is roughly equivalent to writing "errors[0, i, :]" in Matlab   
    var errorsForDegree = errors.ExtractSubArrayShallow(0, i, -1).To1DArray();     
 
    // some formatting of the plot data for the convergence study      
    PlotFormat format = new PlotFormat(lineColor: (LineColors)(i+1),     
                                       Style: Styles.LinesPoints,       
                                       pointType: PointTypes.Diamond,       
                                       pointSize: 2.0);     
 
    // Create log-log plot mesh size vs. error for dgDegrees[i]    
    gpUpwind.PlotLogXLogY(sizes, errorsForDegree,      
                          "Upwind, order " + dgDegrees[i], format);     
}

In [52]:
gpUpwind.PlotNow() // perform the plotting

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0000001 
 
 
 
 
 0.0000010 
 
 
 
 
 0.0000100 
 
 
 
 
 0.0001000 
 
 
 
 
 0.0010000 
 
 
 
 
 0.0100000 
 
 
 
 
 0.1000000 
 
 
 
 
 1.0000000 
 
 
 
 
 0.1 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 Upwind, order 0 
 
 
 Upwind, order 0 
 
 
 
 
 
 
 
 
 
 
 Upwind, order 1 
 
 
 Upwind, order 1 
 
 
 
 
 
 
 
 
 
 
 Upwind, order 2 
 
 
 Upwind, order 2 
 
 
 
 
 
 
 
 
 
 
 Upwind, order 3 
 
 
 Upwind, order 3

In [42]:
/// The convergence plot is also done for the Lax-Friedrichs flux.

In [43]:
var gpLaxFriedrichs = new Gnuplot();     
gpLaxFriedrichs.SetYRange(Math.Pow(10,-7), Math.Pow(10,0));

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


In [44]:
for (int i = 0; i < dgDegrees.Length; i++) {     
 
    var errorsForDegree = errors.ExtractSubArrayShallow(1, i, -1).To1DArray();     
 
    PlotFormat format = new PlotFormat(lineColor: (LineColors)(i+1),     
                                       Style: Styles.LinesPoints,       
                                       pointType: PointTypes.Diamond,       
                                       pointSize: 2.0);     
 
    gpLaxFriedrichs.PlotLogXLogY(sizes, errorsForDegree,      
                                 "LaxFriedrichs, order " + dgDegrees[i],      
                                 format);     
}

In [53]:
gpLaxFriedrichs.PlotNow()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0000001 
 
 
 
 
 0.0000010 
 
 
 
 
 0.0000100 
 
 
 
 
 0.0001000 
 
 
 
 
 0.0010000 
 
 
 
 
 0.0100000 
 
 
 
 
 0.1000000 
 
 
 
 
 1.0000000 
 
 
 
 
 0.1 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 LaxFriedrichs, order 0 
 
 
 LaxFriedrichs, order 0 
 
 
 
 
 
 
 
 
 
 
 LaxFriedrichs, order 1 
 
 
 LaxFriedrichs, order 1 
 
 
 
 
 
 
 
 
 
 
 LaxFriedrichs, order 2 
 
 
 LaxFriedrichs, order 2 
 
 
 
 
 
 
 
 
 
 
 LaxFriedrichs, order 3 
 
 
 LaxFriedrichs, order 3

In [46]:
/// %

In [47]:
/// %==========================================
/// \subsection{Linear regression by ordinary least squares}\label{sec:NumFlux_slopes}
/// \subsection{Linear regression by ordinary least squares}\label{sec:flux_and_convergence_slopes}
/// %===========================================

In [48]:
/// For the determination of the experimental order of convergence (EOC) the linear regression is used 
/// to compute the slope of the line in the log-log plot. In the following, the ordinary least squares method 
/// is implemented to estimate the regression coefficient of the slope for given sets of x- and y-values.

In [49]:
Func<double[], double[], double> slope =      
    delegate(double[] xValues, double[] yValues) {     
 
    if (xValues.Length != yValues.Length) {     
        throw new ArgumentException();     
    }     
 
    xValues = xValues.Select(s => Math.Log10(s)).ToArray();     
    yValues = yValues.Select(s => Math.Log10(s)).ToArray();     
 
    double xAverage = xValues.Sum() / xValues.Length;     
    double yAverage = yValues.Sum() / yValues.Length;     
 
    double v1 = 0.0;     
    double v2 = 0.0;     
 
    // Computation of the regression coefficient for the slope     
    for (int i = 0; i < yValues.Length; i++) {     
        v1 += (xValues[i] - xAverage) * (yValues[i] - yAverage);     
        v2 += Math.Pow(xValues[i] - xAverage, 2);     
    }     
    return v1 / v2;     
};

In [50]:
// Verification that the slope is computed correctly     
Math.Abs(slope(sizes, new double[] { 64.0, 16.0, 4.0, 1.0 }) - 2.0) < 1e-14

True

In [51]:
/// The experimental orders of convergence (EOC) are computed for both fluxes 
/// for each polynomial degree of the ansatzfunctions
for (int i = 0; i < dgDegrees.Length; i++) {     
 
    Console.WriteLine();     
 
    var errorsForDegree = errors.ExtractSubArrayShallow(0, i, -1).To1DArray();     
 
    Console.WriteLine(     
        "Upwind flux, order {0}:\t\t {1:F2}", i,      
        slope(sizes, errorsForDegree));     
 
    errorsForDegree = errors.ExtractSubArrayShallow(1, i, -1).To1DArray();     
 
    Console.WriteLine(     
        "Lax-Friedrichs flux, order {0}:\t {1:F2}", i,     
        slope(sizes, errorsForDegree));     
}


Upwind flux, order 0:		 0,79
Lax-Friedrichs flux, order 0:	 0,73

Upwind flux, order 1:		 1,98
Lax-Friedrichs flux, order 1:	 2,00

Upwind flux, order 2:		 2,96
Lax-Friedrichs flux, order 2:	 2,92

Upwind flux, order 3:		 3,95
Lax-Friedrichs flux, order 3:	 3,97
